In [ ]:
from pyroSAR import identify
name = 'data/download/S1A_IW_GRDH_1SDV_20210206T052511_20210206T052536_036465_0447F4_09CA.zip'
scene = identify(name)
print(scene)

In [ ]:
from pyroSAR.auxdata import dem_autoload
from spatialist import Vector

site = 'data/aoi/aoi_tmp/aoi_tmp.shp'
vrt = 'data/mosaic.vrt'

with Vector(site) as vec:
    vrt = dem_autoload(geometries=[vec],
                       demType='SRTM 1Sec HGT',
                       vrt=vrt,
                       buffer=0.1)


In [ ]:
from pyroSAR.snap import geocode
geocode(infile='data/download/S1A_IW_GRDH_1SDV_20210206T052511_20210206T052536_036465_0447F4_09CA.zip',
        outdir='data/out', test=True)

In [ ]:
from pyroSAR import Archive
dbfile = 'data/scenes.db'
with Archive(dbfile) as archive:
    archive.insert(scene)


In [ ]:
from spatialist import Vector
archive = Archive('data/scenes.db')
outdir = 'data/out'
maxdate = '20220202'
site_shp = 'data/aoi/aoi_tmp/aoi_tmp.shp'
with Vector(site_shp) as site:
    selection_proc = archive.select(vectorobject=site,
                                    processdir=outdir,
                                    maxdate=maxdate,
                                    sensor=('S1A', 'S1B'),
                                    product='GRD',
                                    acquisition_mode='IW',
                                    vv=1)
archive.close()

In [ ]:
# test groupsize, see what grousize is needed to get all steps into one group; currently it's 8 steps + read/write == groupsize of 8
# geocode(infile='data/download/S1A_IW_GRDH_1SDV_20210206T052511_20210206T052536_036465_0447F4_09CA.zip',
#         outdir='data/out',
#         spacing=10,
#         shapefile=site_shp,
#         polarizations=['VV', 'VH'],
#         scaling='dB',
#         cleanup=True,
#         tmpdir='data/tmp',
#         groupsize=8,
#         test=True)

from pyroSAR.snap.auxil import groupbyWorkers
groupbyWorkers('data/out/raw/S1A__IW___A_20210209T170054_Cal_NR_Orb_TF_Spk_TC_dB_proc.xml', n=8)

In [ ]:
from pyroSAR.snap import geocode
import os


files = [os.path.join('data/download/', each) for each in os.listdir('data/download/') if each.endswith('.zip')]
# satfiles = [os.path.join(root, name)
#             for root, dirs, files in os.walk('../sen1ost/data/download/SAR/GRD/2021')
#             for name in files
#             if name.endswith("zip")]
# satfiles = satfiles[0:3]
# site_shp = 'data/aoi/aoi_tmp/aoi_tmp.shp'
site_shp = 'data/aoi/flotow_shape_tmp.geojson'

for single_file in files:
# for single_file in satfiles:
    geocode(infile=single_file,
        outdir='data/out/raw',
        spacing=10,
        shapefile=site_shp,
        polarizations=['VV', 'VH'],
        scaling='dB',
        geocoding_type='Range-Doppler',
        removeS1BorderNoise=False,
        removeS1ThermalNoise=False, #not really necessary for SAR data
        allow_RES_OSV=False,
        demName='SRTM 1Sec HGT',
        externalDEMFile=None,
        terrainFlattening=True,
        speckleFilter='Refined Lee',
        refarea='gamma0', #['sigma0','gamma0']
        test=False,
        cleanup=True,
        tmpdir='data/tmp',
        groupsize=8)



In [ ]:
import rasterio
import geopandas as gpd
from rasterio.mask import mask
import os

tif_files = [file for file in os.listdir('data/out/raw') if file.endswith(".tif")]
# site_shp = 'data/aoi/flotow.shp'
site_shp = 'data/aoi/flotow_shape_tmp.geojson'

# Open the .tif file
for file in tif_files:
    with rasterio.open(os.path.join('data/out/raw', file)) as src:
        # Open the .shp file using geopandas
        shapefile = gpd.read_file(site_shp)

        # Clip the .tif file using the .shp file geometry
        clipped, transform = mask(src, shapefile.geometry, crop=True)
        out_profile = src.profile.copy()
        out_profile.update({'width': clipped.shape[2],'height': clipped.shape[1], 'transform': transform})

        # Save the clipped .tif file
        with rasterio.open(os.path.join('data/out/cut', file), 'w', **out_profile) as dst:
            dst.write(clipped)


In [2]:
#download sen1 images
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import json

username = json.load(open('login_data.json'))['username']
pw = json.load(open('login_data.json'))['pw']
site = 'data/aoi/flotow_shape_tmp.geojson'

api = SentinelAPI(username, pw)

In [3]:
footprint = geojson_to_wkt(read_geojson(site))
products = api.query(
    footprint,
    producttype="GRD",
    date=("20230201", "20230228"),
    platformname="Sentinel-1",
)

products_df = api.to_dataframe(products)
products_df = products_df.sort_values('ingestiondate')
products_df

,title,link,link_alternative,link_icon,summary,ondemand,beginposition,endposition,ingestiondate,missiondatatakeid,...,platformname,platformidentifier,instrumentname,instrumentshortname,productclass,polarisationmode,acquisitiontype,gmlfootprint,footprint,uuid
607791c9-ccf9-4825-870e-85fe5100cd68,S1A_IW_GRDH_1SDV_20230203T051714_20230203T0517...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2023-02-03T05:17:14.017Z, Instrument: SA...",false,2023-02-03 05:17:14.017,2023-02-03 05:17:39.016,2023-02-03 06:12:46.655,370019,...,Sentinel-1,2014-016A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,VV VH,NOMINAL,"<gml:Polygon srsName=""http://www.opengis.net/g...","MULTIPOLYGON (((16.150999 52.16898, 16.665062 ...",607791c9-ccf9-4825-870e-85fe5100cd68
fc19c69e-e588-4994-b031-a2c52a4ff61f,S1A_IW_GRDH_1SDV_20230206T165255_20230206T1653...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2023-02-06T16:52:55.553Z, Instrument: SA...",false,2023-02-06 16:52:55.553,2023-02-06 16:53:20.553,2023-02-06 17:44:22.628,370454,...,Sentinel-1,2014-016A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,VV VH,NOMINAL,"<gml:Polygon srsName=""http://www.opengis.net/g...","MULTIPOLYGON (((11.66927 52.93222, 15.577946 5...",fc19c69e-e588-4994-b031-a2c52a4ff61f
842b3801-8e10-48a3-ace3-9a6213d5290a,S1A_IW_GRDH_1SDV_20230208T052522_20230208T0525...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2023-02-08T05:25:22.289Z, Instrument: SA...",false,2023-02-08 05:25:22.289,2023-02-08 05:25:47.288,2023-02-08 06:16:33.544,370653,...,Sentinel-1,2014-016A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,VV VH,NOMINAL,"<gml:Polygon srsName=""http://www.opengis.net/g...","MULTIPOLYGON (((14.189015 52.454933, 14.709309...",842b3801-8e10-48a3-ace3-9a6213d5290a
5441eb56-0f6b-4f1f-ad01-f1a878e01fa3,S1A_IW_GRDH_1SDV_20230211T170105_20230211T1701...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2023-02-11T17:01:05.313Z, Instrument: SA...",false,2023-02-11 17:01:05.313,2023-02-11 17:01:30.310,2023-02-11 17:48:22.390,371079,...,Sentinel-1,2014-016A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,VV VH,NOMINAL,"<gml:Polygon srsName=""http://www.opengis.net/g...","MULTIPOLYGON (((9.695061 52.698689, 13.574968 ...",5441eb56-0f6b-4f1f-ad01-f1a878e01fa3
985326fd-3f1e-43ee-85b2-8a74b9fff5b8,S1A_IW_GRDH_1SDV_20230215T051713_20230215T0517...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2023-02-15T05:17:13.4Z, Instrument: SAR-...",false,2023-02-15 05:17:13.400,2023-02-15 05:17:38.398,2023-02-15 06:12:32.722,371518,...,Sentinel-1,2014-016A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,VV VH,NOMINAL,"<gml:Polygon srsName=""http://www.opengis.net/g...","MULTIPOLYGON (((16.153543 52.16888, 16.667643 ...",985326fd-3f1e-43ee-85b2-8a74b9fff5b8
2b68365a-fc83-4fa1-a76b-ace5f6f7be8a,S1A_IW_GRDH_1SDV_20230218T165255_20230218T1653...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2023-02-18T16:52:55.246Z, Instrument: SA...",false,2023-02-18 16:52:55.246,2023-02-18 16:53:20.246,2023-02-18 17:40:36.641,371969,...,Sentinel-1,2014-016A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,VV VH,NOMINAL,"<gml:Polygon srsName=""http://www.opengis.net/g...","MULTIPOLYGON (((11.669201 52.932167, 15.577866...",2b68365a-fc83-4fa1-a76b-ace5f6f7be8a
9fa361b3-e8c1-45b8-9708-9a0827184250,S1A_IW_GRDH_1SDV_20230220T052522_20230220T0525...,https://apihub.copernicus.eu/apihub/odata/v1/P..

In [ ]:
api.download_all(products_df.index, directory_path = 'data/download')
api.download()


In [ ]:
test = products_df.head(1)
api.download_all(test.index, directory_path = 'data/download')